<a href="https://colab.research.google.com/github/dyahrosita/Project/blob/main/Project_JTI_5AO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import LIbrary

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from functools import reduce
import datetime as dt

from google.colab import files
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## Initial Audit (week 1)

In [ ]:
# ambil data dari GDrive
IA = pd.read_excel('gdrive/My Drive/JACOB/Data Clean JTI-Jacob 2021 Week 1-IA.xlsx','Data Clean LHHT')
# ganti data yang memiliki nilai N/A dengan 0
IA = IA.fillna(0)

In [ ]:
# hitung total stol
IA['Stock'] = IA['Shop/Front Stock']+IA['Other Stock']
#buat kolom unique name utk mapping (concatenate) dengan menggabungkan store code dengan product description
IA.insert(0, 'Concatenate', IA['SMS Id'].astype('str')+IA['Product Description'])

In [ ]:
# menghapus kolom2 yang tidak dibutuhkan
IA = IA.drop(['Country Id', 'Country Name', 'Auditor Id', 'Resource Name','Instruction Id', 
            'Category Id', 'Category Name', 'Module Name','Entity Id','Original Fact Value',
            'Cluster Id','alternate audit methodology','Shop/Front Stock','Other Stock',
            'Audit Status','Sales'], axis=1)
# mengganti nama kolom agar sesuai dengan yg diinginkan
IA.columns = IA.columns.str.replace('Number Of Observation by Type of Observation','RBP')

In [ ]:
# melihat sekilas data Initial Audit yang sudah dirapihkan
IA.head()

,Concatenate,SMS Id,Bar Code,Product Description,Visit Date,RBP,RSP/Price,Stock
0,1609899Djarum Istimewa Soft Coklat Plain 12 St...,1609899,8991906101651,Djarum Istimewa Soft Coklat Plain 12 Stc (Sktb),2021-07-26,11500,13000,10.0
1,1609986Gudang Garam Red Soft Kspl 12 Stc (Sktb),1609986,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-07-26,12500,13000,20.0
2,1610014Dji Sam Soe Soft Plain 12 Stc (Sktb),1610014,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-07-26,16500,18000,4.0
3,1610908Dji Sam Soe Soft Plain 12 Stc (Sktb),1610908,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-07-26,16500,18000,5.0
4,1610908Panamas 1 Soft Plain 12 Stc (Sktb),1610908,8999909039124,Panamas 1 Soft Plain 12 Stc (Sktb),2021-07-26,12500,14000,10.0


In [ ]:
# membuat format untuk di merge dengan table lainnya
m_IA = IA.copy()
# mengganti nama kolom agar sesuai dengan yg diinginkan
m_IA.columns = m_IA.columns.str.replace('Visit Date', 'IA').str.replace('RBP', 'RBP_IA').str.replace('RSP/Price','RSP_IA').str.replace('Stock','Stock_IA')
m_IA.columns

Index(['Concatenate', 'SMS Id', 'Bar Code', 'Product Description', 'IA',
       'RBP_IA', 'RSP_IA', 'Stock_IA'],
      dtype='object')

## Retail Audit 1 (week 2)

In [ ]:
RA1 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 2 (RA1).xlsx','Data Clean')
RA1 = RA1.fillna(0)

In [ ]:
RA1['Visit Date'] = RA1['Visit Date'].str.replace('Agu','Aug')
RA1['Visit Date'] = pd.to_datetime(RA1['Visit Date'])

In [ ]:
RA1['Stock'] = RA1['Shop/Front Stock']+RA1['Other Stock']
RA1['Purchase'] = RA1['Direct Purchase(Pur1)']+RA1['Missing invoice(Pur3)']+RA1['Missing Stock']-RA1['Double Count']-RA1['Goods Returned']
RA1.insert(0, 'Concatenate', RA1['SMS Id'].astype('str')+RA1['Product Description'])

In [ ]:
RA1 = RA1.drop(['Country Id', 'Country Name', 'Auditor Id', 'Resource Name','Instruction Id', 
            'Category Id', 'Category Name', 'Module Name','Entity Id','Original Fact Value',
            'Cluster Id','alternate audit methodology','Shop/Front Stock','Other Stock',
            'Direct Purchase(Pur1)','Double Count', 'Goods Returned', 'Missing invoice(Pur3)',
            'Missing Stock','Audit Status'], axis=1)
RA1.columns = RA1.columns.str.replace('Number Of Observation by Type of Observation','RBP')

In [ ]:
'''new_IA = IA[['SMS Id','Visit Date']].drop_duplicates()
new_IA.columns = new_IA.columns.str.replace('Visit Date', 'Prev_Visit Date')
new2_IA = IA[['Concatenate','Stock']]
new2_IA.columns = new2_IA.columns.str.replace('Stock', 'Prev_Stock')'''

"new_IA = IA[['SMS Id','Visit Date']].drop_duplicates()\nnew_IA.columns = new_IA.columns.str.replace('Visit Date', 'Prev_Visit Date')\nnew2_IA = IA[['Concatenate','Stock']]\nnew2_IA.columns = new2_IA.columns.str.replace('Stock', 'Prev_Stock')"

In [ ]:
'''RA1 = RA1.merge(new_IA, on='SMS Id')
RA1 = RA1.merge(new2_IA, on='Concatenate')'''

"RA1 = RA1.merge(new_IA, on='SMS Id')\nRA1 = RA1.merge(new2_IA, on='Concatenate')"

In [ ]:
#RA1['Recheck_Sales'] = RA1['Prev_Stock']+RA1['Purchase']-RA1['Stock']

In [ ]:
#RA1[RA1['Recheck_Sales']-RA1['Sales']!=0]

In [ ]:
# membuat format untuk di merge dengan table lainnya
m_RA1 = RA1.copy()
# mengganti nama kolom agar sesuai dengan yg diinginkan
m_RA1.columns = m_RA1.columns.str.replace('Visit Date', 'RA1').str.replace('RBP', 'RBP_RA1').str.replace('RSP/Price','RSP_RA1').str.replace('Stock','Stock_RA1').str.replace('Purchase','Purchase_RA1')
m_RA1.columns

Index(['Concatenate', 'SMS Id', 'Bar Code', 'Product Description', 'RA1',
       'RBP_RA1', 'RSP_RA1', 'Sales', 'Stock_RA1', 'Purchase_RA1'],
      dtype='object')

## Retail Audit 2 (week 3)

In [ ]:
RA2 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 3 (RA2).xlsx','Data Clean')
RA2 = RA2.fillna(0)

In [ ]:
RA2['Visit Date'] = RA2['Visit Date'].str.replace('Agu','Aug')
RA2['Visit Date'] = pd.to_datetime(RA2['Visit Date'])

In [ ]:
RA2.head()

,Country Id,Country Name,Auditor Id,Resource Name,SMS Id,Instruction Id,Category Id,Category Name,Module Name,Entity Id,Bar Code,Product Description,Visit Date,Audit Status,Original Fact Value,Cluster Id,alternate audit methodology,Number Of Observation by Type of Observation,RSP/Price,PREIVIOUS STOCK,Shop/Front Stock,Other Stock,Direct Purchase(Pur1),Double Count,Goods Returned,Missing invoice(Pur3),Missing Stock,Sales,Sales_W2,Item Comment
0,360,INDONESIA,316595,Supri Yanto,1610063,122780577,8,Cigarette,Cigarette,80228,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-08-09,AUDIT COMPLETE,(blank),742047,0.0,15800.0,18000.0,20.0,20.0,0.0,0.0,0.0,0.0,10.0,0.0,10,10.0,0
1,360,INDONESIA,316595,Supri Yanto,1610063,122780577,8,Cigarette,Cigarette,80255,8999909039124,Panamas 1 Soft Plain 12 Stc (Sktb),2021-08-09,AUDIT COMPLETE,(blank),742047,0.0,11900.0,13000.0,10.0,10.0,0.0,0.0,0.0,0.0,10.0,0.0,10,20.0,COVID - Sales turun karena kekurangan pasokan
2,360,INDONESIA,316595,Supri Yanto,1610063,122780577,8,Cigarette,Cigarette,80271,8991906101651,Djarum Istimewa Soft Coklat Plain 12 Stc (Sktb),2021-08-09,AUDIT COMPLETE,(blank),742047,0.0,12000.0,14000.0,20.0,20.0,0.0,0.0,0.0,0.0,10.0,0.0,10,10.0,0
3,360,INDONESIA,316595,Supri Yanto,1610063,122780577,8,Cigarette,Cigarette,80322,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-08-09,AUDIT COMPLETE,(blank),742047,0.0,12200.0,14000.0,20.0,20.0,0.0,0.0,0.0,0.0,10.0,0.0,10,10.0,0
4,360,INDONESIA,316595,Supri Yanto,1610063,122780577,8,Cigarette,Cigarette,86810,8997234650021,Dados Brown Soft Plain 12 Stc (Sktb),2021-08-09,AUDIT COMPLETE,(blank),742047,0.0,5500.0,6500.0,20.0,20.0,0.0,0.0,0.0,0.0,50.0,0.0,50,20.0,"COVID - Sales naik, banyak permintaan produk/k..."


In [ ]:
RA2['Stock'] = RA2['Shop/Front Stock']+RA2['Other Stock']
RA2['Purchase'] = RA2['Direct Purchase(Pur1)']+RA2['Missing invoice(Pur3)']+RA2['Missing Stock']-RA2['Double Count']-RA2['Goods Returned']
RA2.insert(0, 'Concatenate', RA2['SMS Id'].astype('str')+RA2['Product Description'])

In [ ]:
RA2 = RA2.drop(['Country Id', 'Country Name', 'Auditor Id', 'Resource Name','Instruction Id', 
            'Category Id', 'Category Name', 'Module Name','Entity Id','Original Fact Value',
            'Cluster Id','alternate audit methodology','Shop/Front Stock','Other Stock',
            'Direct Purchase(Pur1)','Double Count', 'Goods Returned', 'Missing invoice(Pur3)',
            'Missing Stock','PREIVIOUS STOCK','Audit Status'], axis=1)
RA2.columns = RA2.columns.str.replace('Number Of Observation by Type of Observation','RBP').str.replace('Sales_W2','Claimed_Sales_RA1(di_RA2)').str.replace('Item Comment','Comment_RA2')

In [ ]:
'''new_RA1 = RA1[['SMS Id','Visit Date']].drop_duplicates()
new_RA1.columns = new_RA1.columns.str.replace('Visit Date', 'Prev_Visit Date')
new2_RA1 = RA1[['Concatenate','Stock']]
new2_RA1.columns = new2_RA1.columns.str.replace('Stock', 'Prev_Stock')'''

"new_RA1 = RA1[['SMS Id','Visit Date']].drop_duplicates()\nnew_RA1.columns = new_RA1.columns.str.replace('Visit Date', 'Prev_Visit Date')\nnew2_RA1 = RA1[['Concatenate','Stock']]\nnew2_RA1.columns = new2_RA1.columns.str.replace('Stock', 'Prev_Stock')"

In [ ]:
'''RA2 = RA2.merge(new_RA1, on='SMS Id')
RA2 = RA2.merge(new2_RA1, on='Concatenate')'''

"RA2 = RA2.merge(new_RA1, on='SMS Id')\nRA2 = RA2.merge(new2_RA1, on='Concatenate')"

In [ ]:
#RA2['Recheck_Sales'] = RA2['Prev_Stock']+RA2['Purchase']-RA2['Stock']

In [ ]:
#RA2[RA2['Recheck_Sales']-RA2['Sales']!=0]

In [ ]:
# membuat format untuk di merge dengan table lainnya
m_RA2 = RA2.copy()
# mengganti nama kolom agar sesuai dengan yg diinginkan
m_RA2.columns = m_RA2.columns.str.replace('Visit Date', 'RA2').str.replace('RBP', 'RBP_RA2').str.replace('RSP/Price','RSP_RA2').str.replace('Stock','Stock_RA2').str.replace('Purchase','Purchase_RA2')
m_RA2.shape

(1749, 12)

## Retail Audit 3 (week 4)

In [ ]:
RA3 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 4 (RA3).xlsx','Data Clean')
RA3 = RA3.fillna(0)

In [ ]:
RA3['Visit Date'] = RA3['Visit Date'].str.replace('Agu','Aug')
RA3['Visit Date'] = pd.to_datetime(RA3['Visit Date'])

In [ ]:
RA3['Stock'] = RA3['Shop/Front Stock']+RA3['Other Stock']
RA3['Purchase'] = RA3['Direct Purchase(Pur1)']+RA3['Missing invoice(Pur3)']+RA3['Missing Stock']-RA3['Double Count']-RA3['Goods Returned']
RA3.insert(0, 'Concatenate', RA3['SMS Id'].astype('str')+RA3['Product Description'])

In [ ]:
RA3.head()

,Concatenate,Country Id,Country Name,Auditor Id,Resource Name,SMS Id,Instruction Id,Category Id,Category Name,Module Name,Entity Id,Bar Code,Product Description,Visit Date,Audit Status,Original Fact Value,Cluster Id,alternate audit methodology,Price To Retailer,RSP/Price,PREIVIOUS STOCK,Shop/Front Stock,Other Stock,Direct Purchase(Pur1),Double Count,Goods Returned,Missing invoice(Pur3),Missing Stock,Sales,Sales_W3,Sales_W2,Item Comment,Stock,Purchase
0,1610063Dji Sam Soe Soft Plain 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1610063,122797406,8,Cigarette,Cigarette,80228,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-08-16,AUDIT COMPLETE,(blank),742047,0.0,15800.0,18000.0,20.0,20.0,0.0,0.0,0.0,0.0,10.0,0.0,10,10.0,10.0,0,20.0,10.0
1,1610063Panamas 1 Soft Plain 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1610063,122797406,8,Cigarette,Cigarette,80255,8999909039124,Panamas 1 Soft Plain 12 Stc (Sktb),2021-08-16,AUDIT COMPLETE,(blank),742047,0.0,11900.0,13000.0,10.0,10.0,0.0,0.0,0.0,0.0,20.0,0.0,20,10.0,20.0,"COVID - Sales naik, banyak permintaan produk/k...",10.0,20.0
2,1610063Djarum Istimewa Soft Coklat Plain 12 St...,360,INDONESIA,316595,Supri Yanto,1610063,122797406,8,Cigarette,Cigarette,80271,8991906101651,Djarum Istimewa Soft Coklat Plain 12 Stc (Sktb),2021-08-16,AUDIT COMPLETE,(blank),742047,0.0,12000.0,14000.0,20.0,20.0,0.0,0.0,0.0,0.0,10.0,0.0,10,10.0,10.0,0,20.0,10.0
3,1610063Gudang Garam Red Soft Kspl 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1610063,122797406,8,Cigarette,Cigarette,80322,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-08-16,AUDIT COMPLETE,(blank),742047,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,10.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",0.0,0.0
4,1610063Dados Brown Soft Plain 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1610063,122797406,8,Cigarette,Cigarette,86810,8997234650021,Dados Brown Soft Plain 12 Stc (Sktb),2021-08-16,AUDIT COMPLETE,(blank),742047,0.0,5500.0,6500.0,20.0,20.0,0.0,0.0,0.0,0.0,50.0,0.0,50,50.0,20.0,0,20.0,50.0


In [ ]:
RA3 = RA3.drop(['Country Id', 'Country Name', 'Auditor Id', 'Resource Name','Instruction Id', 
            'Category Id', 'Category Name', 'Module Name','Entity Id','Original Fact Value',
            'Cluster Id','alternate audit methodology','Shop/Front Stock','Other Stock',
            'Direct Purchase(Pur1)','Double Count', 'Goods Returned', 'Missing invoice(Pur3)',
            'Missing Stock','PREIVIOUS STOCK','Audit Status'], axis=1)
RA3.columns = RA3.columns.str.replace(
    'Price To Retailer','RBP').str.replace(
        'Sales_W2','Claimed_Sales_RA1(di_RA3)').str.replace(
            'Sales_W3','Claimed_Sales_RA2(di_RA3)').str.replace(
                'Item Comment','Comment_RA3')

In [ ]:
'''new_RA2 = RA2[['SMS Id','Visit Date']].drop_duplicates()
new_RA2.columns = new_RA2.columns.str.replace('Visit Date', 'Prev_Visit Date')
new2_RA2 = RA2[['Concatenate','Stock']]
new2_RA2.columns = new2_RA2.columns.str.replace('Stock', 'Prev_Stock')'''

"new_RA2 = RA2[['SMS Id','Visit Date']].drop_duplicates()\nnew_RA2.columns = new_RA2.columns.str.replace('Visit Date', 'Prev_Visit Date')\nnew2_RA2 = RA2[['Concatenate','Stock']]\nnew2_RA2.columns = new2_RA2.columns.str.replace('Stock', 'Prev_Stock')"

In [ ]:
'''RA3 = RA3.merge(new_RA2, on='SMS Id')
RA3 = RA3.merge(new2_RA2, on='Concatenate')'''

"RA3 = RA3.merge(new_RA2, on='SMS Id')\nRA3 = RA3.merge(new2_RA2, on='Concatenate')"

In [ ]:
#RA3['Recheck_Sales'] = RA3['Prev_Stock']+RA3['Purchase']-RA3['Stock']

In [ ]:
#RA3[RA3['Recheck_Sales']-RA3['Sales']!=0]

In [ ]:
# membuat format untuk di merge dengan table lainnya
m_RA3 = RA3.copy()
# mengganti nama kolom agar sesuai dengan yg diinginkan
m_RA3.columns = m_RA3.columns.str.replace(
    'Visit Date', 'RA3').str.replace(
        'RBP', 'RBP_RA3').str.replace(
            'RSP/Price','RSP_RA3').str.replace(
                'Stock','Stock_RA3').str.replace(
                    'Purchase','Purchase_RA3')

## Retail Audit 4 (week 5-6)

In [ ]:
RA4 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 6 - CS Version.xlsx','Data Clean')
RA4 = RA4.fillna(0)

In [ ]:
RA4['Stock'] = RA4['Shop/Front Stock']+RA4['Other Stock']
RA4['Purchase'] = RA4['Direct Purchase(Pur1)']+RA4['Missing invoice(Pur3)']+RA4['Missing Stock']-RA4['Double Count']-RA4['Goods Returned']
RA4.insert(0, 'Concatenate', RA4['SMS Id'].astype('str')+RA4['Product Description'])

In [ ]:
RA4.head()

,Concatenate,Country Id,Country Name,Auditor Id,Resource Name,SMS Id,Instruction Id,Category Id,Category Name,Module Name,Entity Id,Bar Code,Product Description,Visit Date,Audit Status,Original Fact Value,Cluster Id,alternate audit methodology,Price To Retailer,RSP/Price,PREIVIOUS STOCK,Shop/Front Stock,Other Stock,Direct Purchase(Pur1),Double Count,Goods Returned,Missing invoice(Pur3),Missing Stock,Sales,Sales_W4&5,Sales_W3,Sales_W2,Item Comment,Stock,Purchase
0,1609895Dji Sam Soe Soft Plain 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1609895,5433835,198107565,CIGARETTE,CIGARETTE,73047810,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-09-02,AUDIT COMPLETE,(blank),742047,0.0,15800.0,18000.0,2.0,1.0,0.0,0.0,0.0,0.0,40.0,0.0,41,21.0,20.0,22.0,"COVID - Sales naik, banyak permintaan produk/k...",1.0,40.0
1,1609895Apache Brown Soft Plain 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1609895,5433835,198107565,CIGARETTE,CIGARETTE,73051466,8994729400178,Apache Brown Soft Plain 12 Stc (Sktb),2021-09-02,AUDIT COMPLETE,(blank),742047,0.0,9000.0,10000.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,9,3.0,10.0,22.0,"COVID - Sales naik, banyak permintaan produk/k...",1.0,10.0
2,1609895Gudang Garam Red Soft Kspl 12 Stc (Sktb),360,INDONESIA,316595,Supri Yanto,1609895,5433835,198107565,CIGARETTE,CIGARETTE,73104371,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-09-02,AUDIT COMPLETE,(blank),742047,0.0,12200.0,13000.0,2.0,2.0,0.0,0.0,0.0,0.0,40.0,0.0,40,30.0,26.0,17.0,"COVID - Sales naik, banyak permintaan produk/k...",2.0,40.0
3,1609895Sampoerna 'A' Hijau Edisi Khusus Box Pl...,360,INDONESIA,316595,Supri Yanto,1609895,5433835,198107565,CIGARETTE,CIGARETTE,73203844,8999909000988,Sampoerna 'A' Hijau Edisi Khusus Box Plain 10+...,2021-09-02,AUDIT COMPLETE,(blank),742047,0.0,11600.0,13000.0,2.0,2.0,0.0,0.0,0.0,0.0,40.0,0.0,40,21.0,21.0,21.0,"COVID - Sales naik, banyak permintaan produk/k...",2.0,40.0
4,1609895Djarum Istimewa Soft Coklat Plain 12 St...,360,INDONESIA,316595,Supri Yanto,1609895,5433835,198107565,CIGARETTE,CIGARETTE,73316560,8991906101651,Djarum Istimewa Soft Coklat Plain 12 Stc (Sktb),2021-09-02,AUDIT COMPLETE,(blank),742047,0.0,12100.0,13000.0,5.0,2.0,0.0,0.0,0.0,0.0,200.0,0.0,203,50.0,98.0,22.0,"COVID - Sales naik, banyak permintaan produk/k...",2.0,200.0


In [ ]:
RA4 = RA4.drop(['Country Id', 'Country Name', 'Auditor Id', 'Resource Name','Instruction Id', 
            'Category Id', 'Category Name', 'Module Name','Entity Id','Original Fact Value',
            'Cluster Id','alternate audit methodology','Shop/Front Stock','Other Stock',
            'Direct Purchase(Pur1)','Double Count', 'Goods Returned', 'Missing invoice(Pur3)',
            'Missing Stock','PREIVIOUS STOCK','Audit Status'], axis=1)
RA4.columns = RA4.columns.str.replace(
    'Price To Retailer','RBP').str.replace(
        'Sales_W2','Claimed_Sales_RA1(di_RA4)').str.replace(
            'Sales_W3','Claimed_Sales_RA2(di_RA4)').str.replace(
                'Sales_W4&5', 'Claimed_Sales_RA3(di_RA4)').str.replace(
                    'Item Comment','Comment_RA4')

In [ ]:
'''new_RA3 = RA3[['SMS Id','Visit Date']].drop_duplicates()
new_RA3.columns = new_RA3.columns.str.replace('Visit Date', 'Prev_Visit Date')
new2_RA3 = RA3[['Concatenate','Stock']]
new2_RA3.columns = new2_RA3.columns.str.replace('Stock', 'Prev_Stock')'''

"new_RA3 = RA3[['SMS Id','Visit Date']].drop_duplicates()\nnew_RA3.columns = new_RA3.columns.str.replace('Visit Date', 'Prev_Visit Date')\nnew2_RA3 = RA3[['Concatenate','Stock']]\nnew2_RA3.columns = new2_RA3.columns.str.replace('Stock', 'Prev_Stock')"

In [ ]:
'''RA4 = RA4.merge(new_RA3, on='SMS Id')
RA4 = RA4.merge(new2_RA3, on='Concatenate')'''

"RA4 = RA4.merge(new_RA3, on='SMS Id')\nRA4 = RA4.merge(new2_RA3, on='Concatenate')"

In [ ]:
#RA4.head()

In [ ]:
#RA4['Recheck_Sales'] = RA4['Prev_Stock']+RA4['Purchase']-RA4['Stock']

In [ ]:
#RA4[RA4['Recheck_Sales']-RA4['Sales']!=0]

In [ ]:
# membuat format untuk di merge dengan table lainnya
m_RA4 = RA4.copy()
# mengganti nama kolom agar sesuai dengan yg diinginkan
m_RA4.columns = m_RA4.columns.str.replace(
    'Visit Date', 'RA4').str.replace(
        'RBP', 'RBP_RA4').str.replace(
            'RSP/Price','RSP_RA4').str.replace(
                'Stock','Stock_RA4').str.replace(
                    'Purchase','Purchase_RA4')

## Merge Data Retail Audit into 1 Dataframe

### Merge DataFrame IA dan RA1

In [ ]:
inner = pd.merge(m_IA, m_RA1[['RA1','RBP_RA1', 'RSP_RA1', 'Stock_RA1', 'Purchase_RA1',
       'Concatenate']], on="Concatenate", how="inner")
left = pd.merge(m_IA, m_RA1[['RA1','RBP_RA1', 'RSP_RA1', 'Stock_RA1', 'Purchase_RA1',
       'Concatenate']], on="Concatenate", how="left", indicator=True)
right = pd.merge(m_IA[['IA','RBP_IA','RSP_IA','Stock_IA','Concatenate']], m_RA1, on="Concatenate", how="right", indicator=True)

In [ ]:
df_merge = inner.append(left[left['_merge']=='left_only'], ignore_index = True)
df_merge = df_merge.append(right[right['_merge']=='right_only'], ignore_index = True)

In [ ]:
df_merge = df_merge.drop('_merge', axis=1)
df_merge.shape

(1924, 14)

### Merge DataFrame IA+RA1 dan RA2

In [ ]:
inner = pd.merge(df_merge, m_RA2[['Concatenate','RA2','RBP_RA2', 'RSP_RA2', 'Stock_RA2', 'Purchase_RA2', 'Claimed_Sales_RA1(di_RA2)', 'Comment_RA2'
       ]], on="Concatenate", how="inner")
left = pd.merge(df_merge, m_RA2[['Concatenate','RA2','RBP_RA2', 'RSP_RA2', 'Stock_RA2', 'Purchase_RA2', 'Claimed_Sales_RA1(di_RA2)', 'Comment_RA2']], on="Concatenate", how="left", indicator=True)
right = pd.merge(df_merge[['Concatenate','IA','RBP_IA','RSP_IA','Stock_IA','RA1','RBP_RA1', 'RSP_RA1', 'Stock_RA1', 'Purchase_RA1']], m_RA2, on="Concatenate", how="right", indicator=True)

In [ ]:
inner.shape

(1693, 21)

In [ ]:
left[left['_merge']=='left_only'].shape

(231, 22)

In [ ]:
right[right['_merge']=='right_only'].shape

(56, 22)

In [ ]:
df_merge = inner.append(left[left['_merge']=='left_only'], ignore_index = True)
df_merge = df_merge.append(right[right['_merge']=='right_only'], ignore_index = True)

In [ ]:
df_merge = df_merge.drop('_merge', axis=1)
df_merge.shape

(1980, 21)

### Merge DataFrame IA+RA1+RA2 dan RA3

In [ ]:
inner = pd.merge(df_merge, 
                 m_RA3[['Concatenate','RA3','RBP_RA3', 'RSP_RA3', 'Stock_RA3', 'Purchase_RA3', 
                        'Claimed_Sales_RA1(di_RA3)', 'Claimed_Sales_RA2(di_RA3)','Comment_RA3']],
                 on="Concatenate", how="inner")
left = pd.merge(df_merge, 
                m_RA3[['Concatenate','RA3','RBP_RA3', 'RSP_RA3', 'Stock_RA3', 'Purchase_RA3', 
                       'Claimed_Sales_RA1(di_RA3)', 'Claimed_Sales_RA2(di_RA3)','Comment_RA3']], 
                on="Concatenate", how="left", indicator=True)
right = pd.merge(df_merge[['Concatenate','IA','RBP_IA','RSP_IA','Stock_IA',
                           'RA1','RBP_RA1', 'RSP_RA1', 'Stock_RA1', 'Purchase_RA1',
                           'RA2','RBP_RA2', 'RSP_RA2', 'Stock_RA2', 'Purchase_RA2','Claimed_Sales_RA1(di_RA2)','Comment_RA2']], 
                 m_RA3, on="Concatenate", how="right", indicator=True)

In [ ]:
df_merge = inner.append(left[left['_merge']=='left_only'], ignore_index = True)
df_merge = df_merge.append(right[right['_merge']=='right_only'], ignore_index = True)

In [ ]:
df_merge.head()

,Concatenate,SMS Id,Bar Code,Product Description,IA,RBP_IA,RSP_IA,Stock_IA,RA1,RBP_RA1,RSP_RA1,Stock_RA1,Purchase_RA1,Sales,RA2,RBP_RA2,RSP_RA2,Stock_RA2,Purchase_RA2,Claimed_Sales_RA1(di_RA2),Comment_RA2,RA3,RBP_RA3,RSP_RA3,Stock_RA3,Purchase_RA3,Claimed_Sales_RA1(di_RA3),Claimed_Sales_RA2(di_RA3),Comment_RA3,_merge
0,1609986Gudang Garam Red Soft Kspl 12 Stc (Sktb),1609986,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-07-26,12500.0,13000.0,20.0,2021-08-02,12500.0,13000.0,3.0,0.0,NaN,2021-08-10,12500.0,13000.0,3.0,17.0,17.0,0,2021-08-18,12500.0,13000.0,3.0,20.0,17.0,17.0,"COVID - Sales naik, banyak permintaan produk/k...",NaN
1,1610014Dji Sam Soe Soft Plain 12 Stc (Sktb),1610014,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-07-26,16500.0,18000.0,4.0,2021-08-02,16500.0,18000.0,4.0,4.0,NaN,2021-08-12,16500.0,17000.0,4.0,0.0,4.0,0,2021-08-19,16500.0,17000.0,4.0,0.0,4.0,0.0,0,NaN
2,1610908Dji Sam Soe Soft Plain 12 Stc (Sktb),1610908,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-07-26,16500.0,18000.0,5.0,2021-08-03,16000.0,18000.0,10.0,10.0,NaN,2021-08-09,16000.0,18000.0,10.0,10.0,5.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,15850.0,18000.0,10.0,30.0,5.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",NaN
3,1610908Panamas 1 Soft Plain 12 Stc (Sktb),1610908,8999909039124,Panamas 1 Soft Plain 12 Stc (Sktb),2021-07-26,12500.0,14000.0,10.0,2021-08-03,12500.0,14000.0,10.0,0.0,NaN,2021-08-09,12500.0,14000.0,10.0,10.0,0.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,12500.0,14000.0,10.0,60.0,0.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",NaN
4,1610908Djarum Istimewa Soft Coklat Plain 12 St...,1610908,8991906101651,Djarum Istimewa Soft Coklat Plain 12 Stc (Sktb),2021-07-26,12500.0,14000.0,10.0,2021-08-03,12500.0,14000.0,10.0,0.0,NaN,2021-08-09,12500.0,14000.0,10.0,10.0,0.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,12500.0,14000.0,10.0,30.0,0.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",NaN


In [ ]:
df_merge = df_merge.drop('_merge', axis=1)
df_merge.shape

(2437, 29)

### Merge DataFrame IA+RA1+RA2+RA3 dan RA4

In [ ]:
inner = pd.merge(df_merge, 
                 m_RA4[['Concatenate','RA4','RBP_RA4', 'RSP_RA4', 'Stock_RA4', 'Purchase_RA4', 
                        'Claimed_Sales_RA1(di_RA4)', 'Claimed_Sales_RA2(di_RA4)','Claimed_Sales_RA3(di_RA4)',
                        'Comment_RA4']],
                 on="Concatenate", how="inner")
left = pd.merge(df_merge, 
                m_RA4[['Concatenate','RA4','RBP_RA4', 'RSP_RA4', 'Stock_RA4', 'Purchase_RA4', 
                        'Claimed_Sales_RA1(di_RA4)', 'Claimed_Sales_RA2(di_RA4)','Claimed_Sales_RA3(di_RA4)',
                        'Comment_RA4']], 
                on="Concatenate", how="left", indicator=True)
right = pd.merge(df_merge[['Concatenate','IA','RBP_IA','RSP_IA','Stock_IA',
                           'RA1','RBP_RA1', 'RSP_RA1', 'Stock_RA1', 'Purchase_RA1',
                           'RA2','RBP_RA2', 'RSP_RA2', 'Stock_RA2', 'Purchase_RA2','Claimed_Sales_RA1(di_RA2)','Comment_RA2',
                           'RA3','RBP_RA3', 'RSP_RA3', 'Stock_RA3', 'Purchase_RA3','Claimed_Sales_RA1(di_RA3)','Claimed_Sales_RA2(di_RA3)','Comment_RA3']],
                 m_RA4, on="Concatenate", how="right", indicator=True)

In [ ]:
df_merge = inner.append(left[left['_merge']=='left_only'], ignore_index = True)
df_merge = df_merge.append(right[right['_merge']=='right_only'], ignore_index = True)

In [ ]:
df_merge.head()

,Concatenate,SMS Id,Bar Code,Product Description,IA,RBP_IA,RSP_IA,Stock_IA,RA1,RBP_RA1,RSP_RA1,Stock_RA1,Purchase_RA1,Sales,RA2,RBP_RA2,RSP_RA2,Stock_RA2,Purchase_RA2,Claimed_Sales_RA1(di_RA2),Comment_RA2,RA3,RBP_RA3,RSP_RA3,Stock_RA3,Purchase_RA3,Claimed_Sales_RA1(di_RA3),Claimed_Sales_RA2(di_RA3),Comment_RA3,RA4,RBP_RA4,RSP_RA4,Stock_RA4,Purchase_RA4,Claimed_Sales_RA1(di_RA4),Claimed_Sales_RA2(di_RA4),Claimed_Sales_RA3(di_RA4),Comment_RA4,ED_RA1,ED_RA2,ED_RA3,ED_RA4
0,1609986Gudang Garam Red Soft Kspl 12 Stc (Sktb),1609986,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-07-26,12500.0,13000.0,20.0,2021-08-02,12500.0,13000.0,3.0,0.0,NaN,2021-08-10,12500.0,13000.0,3.0,17.0,17.0,0,2021-08-18,12500.0,13000.0,3.0,20.0,17.0,17.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-09-03,12200.0,13000.0,2.0,20.0,17.0,17.0,20.0,0,7 days,8 days,8 days,16 days
1,1610908Dji Sam Soe Soft Plain 12 Stc (Sktb),1610908,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-07-26,16500.0,18000.0,5.0,2021-08-03,16000.0,18000.0,10.0,10.0,NaN,2021-08-09,16000.0,18000.0,10.0,10.0,5.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,15850.0,18000.0,10.0,30.0,5.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-09-03,15900.0,18000.0,1.0,40.0,5.0,10.0,30.0,"COVID - Sales naik, banyak permintaan produk/k...",8 days,6 days,9 days,16 days
2,1610908Panamas 1 Soft Plain 12 Stc (Sktb),1610908,8999909039124,Panamas 1 Soft Plain 12 Stc (Sktb),2021-07-26,12500.0,14000.0,10.0,2021-08-03,12500.0,14000.0,10.0,0.0,NaN,2021-08-09,12500.0,14000.0,10.0,10.0,0.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,12500.0,14000.0,10.0,60.0,0.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-09-03,12500.0,14000.0,1.0,10.0,0.0,10.0,60.0,COVID - Sales turun karena kekurangan pasokan,8 days,6 days,9 days,16 days
3,1610908Djarum Istimewa Soft Coklat Plain 12 St...,1610908,8991906101651,Djarum Istimewa Soft Coklat Plain 12 Stc (Sktb),2021-07-26,12500.0,14000.0,10.0,2021-08-03,12500.0,14000.0,10.0,0.0,NaN,2021-08-09,12500.0,14000.0,10.0,10.0,0.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,12500.0,14000.0,10.0,30.0,0.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-09-03,12500.0,14000.0,1.0,10.0,0.0,10.0,30.0,COVID - Sales turun karena kekurangan pasokan,8 days,6 days,9 days,16 days
4,1610908Gudang Garam Red Soft Kspl 12 Stc (Sktb),1610908,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-07-26,12500.0,14000.0,10.0,2021-08-03,12500.0,14000.0,10.0,0.0,NaN,2021-08-09,12500.0,14000.0,10.0,10.0,0.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-08-18,12150.0,14000.0,10.0,30.0,0.0,10.0,"COVID - Sales naik, banyak permintaan produk/k...",2021-09-03,12500.0,14000.0,1.0,20.0,0.0,10.0,30.0,0,8 days,6 days,9 days,16 days


In [ ]:
df_merge = df_merge.drop('_merge', axis=1)
df_merge.shape

(3424, 38)

# Menghitung Sales dengan Elapsed day

In [ ]:
df_merge['ED_RA1'] = df_merge['RA1'] - df_merge['IA']
df_merge['ED_RA2'] = df_merge['RA2'] - df_merge['RA1']
df_merge['ED_RA3'] = df_merge['RA3'] - df_merge['RA2']
df_merge['ED_RA4'] = df_merge['RA4'] - df_merge['RA3']

In [ ]:
df_merge[df_merge['ED_RA1'].isna()]


,Concatenate,SMS Id,Bar Code,Product Description,IA,RBP_IA,RSP_IA,Stock_IA,RA1,RBP_RA1,RSP_RA1,Stock_RA1,Purchase_RA1,Sales,RA2,RBP_RA2,RSP_RA2,Stock_RA2,Purchase_RA2,Claimed_Sales_RA1(di_RA2),Comment_RA2,RA3,RBP_RA3,RSP_RA3,Stock_RA3,Purchase_RA3,Claimed_Sales_RA1(di_RA3),Claimed_Sales_RA2(di_RA3),Comment_RA3,RA4,RBP_RA4,RSP_RA4,Stock_RA4,Purchase_RA4,Claimed_Sales_RA1(di_RA4),Claimed_Sales_RA2(di_RA4),Claimed_Sales_RA3(di_RA4),Comment_RA4,ED_RA1,ED_RA2,ED_RA3,ED_RA4
847,1610126Minak Djinggo Kuning Soft Plain 10 Stc ...,1610126,8993989111008,Minak Djinggo Kuning Soft Plain 10 Stc (Sktb),2021-07-26,8000.0,10000.0,3.0,NaT,NaN,NaN,NaN,NaN,NaN,2021-08-10,8000.0,10000.0,3.0,0.0,0.0,minggu lalu NA,2021-08-17,8000.0,10000.0,3.0,30.0,5.0,10.0,W3 10,2021-09-04,8000.0,9000.0,20.0,50.0,5.0,10.0,30.0,0,NaT,NaT,7 days,18 days
848,1610126Djarum Coklat Soft Plain 12 Stc (Sktb),1610126,8991906101644,Djarum Coklat Soft Plain 12 Stc (Sktb),2021-07-26,13500.0,15000.0,8.0,NaT,NaN,NaN,NaN,NaN,NaN,2021-08-10,13500.0,15000.0,8.0,0.0,0.0,minggu lalu NA,2021-08-17,13500.0,15000.0,8.0,40.0,10.0,20.0,W3 20,2021-09-04,13000.0,14000.0,20.0,60.0,10.0,20.0,40.0,S4U: Pelanggan Baru,NaT,NaT,7 days,18 days
849,1610126Dji Sam Soe Soft Plain 12 Stc (Sktb),1610126,8999909028234,Dji Sam Soe Soft Plain 12 Stc (Sktb),2021-07-26,16000.0,18000.0,15.0,NaT,NaN,NaN,NaN,NaN,NaN,2021-08-10,16000.0,18000.0,15.0,0.0,0.0,minggu lalu NA,2021-08-17,16000.0,18000.0,15.0,60.0,20.0,30.0,W3 30,2021-09-04,15900.0,17500.0,45.0,100.0,20.0,30.0,60.0,S4U: Pelanggan Baru,NaT,NaT,7 days,18 days
850,1610126Gudang Garam Red Soft Kspl 12 Stc (Sktb),1610126,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),2021-07-26,12600.0,14000.0,10.0,NaT,NaN,NaN,NaN,NaN,NaN,2021-08-10,12600.0,14000.0,10.0,0.0,0.0,Minggu lalu NA,2021-08-17,12600.0,14000.0,10.0,0.0,10.0,0.0,0,2021-09-04,12200.0,14000.0,50.0,100.0,10.0,0.0,0.0,baru belanaj,NaT,NaT,7 days,18 days
851,1610126Sampoerna 'A' Hijau Box Plain 12 Stc (S...,1610126,8999909085114,Sampoerna 'A' Hijau Box Plain 12 Stc (Sktb),2021-07-26,11700.0,14000.0,10.0,NaT,NaN,NaN,NaN,NaN,NaN,2021-08-10,11700.0,14000.0,10.0,0.0,0.0,minggu lalu NA,2021-08-17,11700.0,14000.0,10.0,40.0,10.0,20.0,w3 20,2021-09-04,11600.0,13000.0,30.0,70.0,10.0,20.0,40.0,S4U: Pelanggan Baru,NaT,NaT,7 days,18 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,1615038Djarum 76 Brown Box Plain 12 Stc (Sktb),1615038,8991906101668,Djarum 76 Brown Box Plain 12 Stc (Sktb),NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,80.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-05,12500.0,14000.0,5.0,85.0,0.0,0.0,0.0,ok,NaT,NaT,NaT,NaT
3420,1615038Gudang Garam Red Soft Kspl 12 Stc (Sktb),1615038,8998989501125,Gudang Garam Red Soft Kspl 12 Stc (Sktb),NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,21.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-05,12400.0,13500.0,4.0,25.0,0.0,0.0,0.0,ok,NaT,NaT,NaT,NaT
3421,1615038Sampoerna 'A' Hijau Edisi Khusus Box Pl...,1615038,8999909000988,Sampoerna 'A' Hijau Edisi Khusus Box Plain 10+...,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,83.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-05,12500.0,14000.0,2.0,85.0,0.0,0.0,0.0,ok,NaT,NaT,NaT,NaT
3422,1615038Gudang Garam Taman Sriwedari Soft Kspl ...,1615038,8998989520126,Gudang Garam Taman Sriwedari Soft Kspl 12 Stc ...,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,14.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-05,10500.0,12000.0,1.0,15.0,0.0,0.0,0.0,ok,NaT,NaT,NaT,NaT


In [ ]:
df_merge['Sales_RA1'] = (df_merge['Stock_IA']+df_merge['Purchase_RA1']-df_merge['Stock_RA1'])/df_merge['ED_RA1']*7
df_merge['Sales_RA2'] = (df_merge['Stock_RA1']+df_merge['Purchase_RA2']-df_merge['Stock_RA2'])/df_merge['ED_RA2']*7
df_merge['Sales_RA3'] = (df_merge['Stock_RA2']+df_merge['Purchase_RA3']-df_merge['Stock_RA3'])/df_merge['ED_RA3']*7
df_merge['Sales_RA4'] = (df_merge['Stock_RA3']+df_merge['Purchase_RA4']-df_merge['Stock_RA4'])/df_merge['ED_RA4']*7

TypeError: ignored

In [ ]:
# MAPPING AREA, TYPE TOKO, 
# MAPPING PRODUCT DESC -> BRAND
# 

# Store Observation

In [ ]:
drop_column = ['LANGUAGE_ID', 'VERSION', 'RESPONSE_NAME', 'RESPONSE_STATUS', 'SOURCE_OF_ENTRY', 'RESPONSE_MODE',
              'BACKCHK_FLAG', 'QC_STATUS', 'START_TIME', 'SUBMIT_TIME', 'SYNC_TIME', 'SERVER_SYNC_TIME',
              'CREATED_BY', 'CDAR_USER_ID', 'START_TIME_V1', 'SUBMIT_TIME_V1', 'CREATED_BY_V1', 'CDAR_USER_ID_V1',
              'SURVEY_DURATION_V1', 'SURVEY_ID', 'SURVEY_VERSION', 'QC_REMARK', 'QC_START_DATE', 'QC_END_DATE',
              'QC_SYNC_DATE', 'QC_SUBMIT_BY', 'APP_VERSION', 'PLATFORM_NAME', 'PLATFORM_MODEL', 'PLATFORM_VERSION',
              'FV_STATUS', 'FV_REMARK', 'FV_START_DATE', 'FV_END_DATE', 'FV_SYNC_DATE', 'FV_SUBMIT_BY', 'SCC_STATUS',
              'SCC_REMARK', 'SCC_START_DATE', 'SCC_END_DATE', 'SCC_SYNC_DATE', 'SCC_SUBMIT_BY', 'FLAG1', 'FLAG2',
              'FLAG3', 'FLAG4', 'FLAG5', 'SURVEY_MINOR_VERSION', 'Default Question Text', 'Q1. ID Auditor / CT',
              'Q2. Nama Auditor / CT', 'Q3B. Jam Survey', 'Latitude', 'LON_1', 'Q3L_Status FLAG', 'Q3M_Area Code',
              'Q4. Apakah informasi toko sudah benar?', 'Q4A. Apakah informasi nama toko sudah benar?', 'Q4A_1. Nama toko', 
              'Q4B. Apakah informasi alamat toko sudah benar?', 'Q4B_1. Alamat', 'Q4C. Apakah informasi type toko sudah benar?',
              'Q4C_1. Type toko', 'Q4D. Apakah informasi untuk (Provinsi, Kota, Kel, Kec) sudah benar?', 'Q4D_1. Provinsi',
              'Q4E_1. Kota', 'Q4F_1. Kecamatan', 'Q4G_1. Kelurahan', 'Q5A. Apakah ada papan penanda toko / spanduk terdapat Nama Toko?',
              'Q5B. Nama toko (menanyakan ke responden / tetangga)', 'Q5C. Tidak ada nama toko informasi (NA):NA', 'Q6A',
              'Q6. nama toko di penanda toko / toko tanda-tanda / spanduk toko', 'Q8. Lot / Los / Blokir',
              'Q9. informasi tambahan', 'Q10. Nama Lengkap pemilik Toko', 'Q11. Nama Penjaga Toko responden (di-interview)',
              'Q12. nomor telepon - Telkom (area nomor kode + telepon)', 'Q13. Nomor ponsel', 'Alamat', 'Q50N_1', 
              'Q50N_2', 'Q50N_3', 'Q50N_4', 'Q25', 'Q26', 'Q16_3', 'Q16_4', 'Q16_5', 'Q16_6', 'Q16_7', 'Q16_8', 'Q16_9',
              'Q16_10', 'Q16_11', 'Q16_12', 'Kecamatan', 'Kelurahan',
              'Q32_You have completed the survey, please check the data on summary page.']

In [ ]:
Promotion_APM = ['Q50A_Apakah toko anda mendapatkan material promosi (sticker,poster,etc) untuk brand dibawah ini::Brand Apache Merah',
                 'Q50B_Apakah ada promosi diskon untuk Brand di bawah ini ::Brand Apache Merah',
                 'Q50C_Apakah ada promosi Beli berapa pack gratis berapa pack contoh beli 2 gratis 1, beli 4 gratis 1 untuk Brand di bawah ini ::Brand Apache Merah',
                 'Q50D_Apakah ada promosi hadiah penjualan : Dapatkan Poin, Dapatkan Hadiah, dll. untuk Brand di bawah ini ::Brand Apache Merah',
                 'Q50E_Apakah ada promosi tukar cangkang/bungkus untuk Brand di bawah ini ::Brand Apache Merah',
                 'Q50F_Apakah ada promosi bundling produk untuk Brand di bawah ini ::Brand Apache Merah']

Visibility_APM = ['Q50H_Apakah ada POSM sunscreen untuk brand berikut.:Brand Apache Merah',
                  'Q50I_Apakah ada POSM Tape Blocking Display untuk brand berikut.:Brand Apache Merah',
                  'Q50J_Apakah ada POSM decorative pack untuk brand berikut:Brand Apache Merah',
                  'Q50K_Apakah ada POSM sticker per pack atau perbatang untuk brand berikut:Brand Apache Merah',
                  'Q50L_Apakah ada POSM Blocking Display (Dummy Pack ada 5 Pack kosong) untuk brand berikut:Brand Apache Merah',
                  'Q50M_Apakah ada POSM Hanging Lighter untuk brand berikut:Brand Apache Merah']

In [ ]:
MAP = pd.read_excel('gdrive/My Drive/JACOB/Datamap DCT Jacob.xlsx')

## Store Observation - IA

In [ ]:
SO_IA = pd.read_excel('gdrive/My Drive/JACOB/Data Clean JTI-Jacob 2021 Week 1-IA.xlsx','Rawdata DCT')

In [ ]:
col_name = pd.DataFrame(SO_IA.columns, columns=['Variable'])
col_name = col_name.merge(MAP[['Variable','Label']], on='Variable')
col_name = col_name.drop('Variable', axis=1)
SO_IA.columns = col_name['Label']

In [ ]:
SO_IA = SO_IA.drop(drop_column, axis=1)

## Store Observation 1

In [ ]:
SO1 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 2 (RA1).xlsx','Rawdata DCT')

In [ ]:
col_name = pd.DataFrame(SO1.columns, columns=['Variable'])
col_name = col_name.merge(MAP[['Variable','Label']], on='Variable')
col_name = col_name.drop('Variable', axis=1)
SO1.columns = col_name['Label']

In [ ]:
drop_column.append('Q7. Lantai')

In [ ]:
SO1 = SO1.drop(drop_column, axis=1)

## Store Observation 2 (Pre Launch)

In [ ]:
SO2 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 3 (RA2).xlsx','DCT')

In [ ]:
col_name = pd.DataFrame(SO2.columns, columns=['Variable'])
col_name = col_name.merge(MAP[['Variable','Label']], on='Variable')
col_name = col_name.drop('Variable', axis=1)
SO2.columns = col_name['Label']

In [ ]:
SO2 = SO2.drop(drop_column, axis=1)

## Store Observation 3 (Post Launch 1)

In [ ]:
SO3 = pd.read_excel('gdrive/My Drive/JACOB/Data Clean Jacob 2021 Week 4 (RA3).xlsx','DCT')

In [ ]:
col_name = pd.DataFrame(SO3.columns, columns=['Variable'])
col_name = col_name.merge(MAP[['Variable','Label']], on='Variable')
col_name = col_name.drop('Variable', axis=1)
SO3.columns = col_name['Label']

In [ ]:
SO3 = SO3.drop(drop_column, axis=1)

### Merge Store Observation

In [ ]:
mapping_toko = SO_IA[['Store Code','Type Toko','Provinsi']]
mapping_toko.shape

In [ ]:
mapping_toko = mapping_toko.append(SO1[['Store Code','Type Toko','Provinsi']])
mapping_toko.shape

In [ ]:
mapping_toko = mapping_toko.append(SO2[['Store Code','Type Toko','Provinsi']])
mapping_toko.shape

In [ ]:
df_merge = inner.append(left[left['_merge']=='left_only'], ignore_index = True)
df_merge = df_merge.append(right[right['_merge']=='right_only'], ignore_index = True)

In [ ]:
df_merge = df_merge.drop('_merge', axis=1)
df_merge.shape

# Notes

sales = prev_stock+purchase+missinginv+missinstock-shopstock-otherstock - goodreturn-wholesales



# Checking

##1. Ada Promotion tapi tidak ada sales

##2. Ada POSM tapi tidak ada sales

##3. Ada sales tapi tidak ada promotion ataupun POSM